In [2]:
# Setup Jupyter Dash
from jupyter_dash import JupyterDash

# Dash components
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output

# Charting
import plotly.express as px

# Data + utilities
import pandas as pd
import base64

# Import my CRUD module from Project One
from CRUD_Python_Module import AnimalShelter


# ============================================================
# DATABASE CONNECTION (Model)
# ============================================================

# Required login credentials for the project
username = "aacuser"
password = "YourPasswordHere"

# Connect to MongoDB using my CRUD class
db = AnimalShelter(username, password)

# Pull ALL data initially (projection removes _id to avoid Dash errors)
df = pd.DataFrame.from_records(db.read({}, {"_id": 0}))

# If no data is returned, avoid crashing
if df is None or df.empty:
    df = pd.DataFrame([])


# ============================================================
# DASHBOARD LAYOUT (View)
# ============================================================

app = JupyterDash(__name__)

# Add Grazioso Salvare logo
image_filename = "Grazioso Salvare Logo.png"
encoded_image = base64.b64encode(open(image_filename, "rb").read()).decode()

logo_img = html.Img(
    src=f"data:image/png;base64,{encoded_image}",
    style={"height": "120px"}
)

app.layout = html.Div([

    # Header section with logo + identifier
    html.Div([
        logo_img,
        html.Div([
            html.H2("Grazioso Salvare Rescue Candidate Dashboard"),
            html.H4("America Sanchez-Garcia")
        ])
    ], style={"display": "flex", "gap": "20px", "alignItems": "center"}),

    html.Hr(),

    # ========================================================
    # FILTER OPTIONS (Controller input)
    # ========================================================
    html.Div([
        html.H4("Select Rescue Type"),
        dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": "Reset (All Animals)", "value": "reset"},
                {"label": "Water Rescue", "value": "water"},
                {"label": "Mountain or Wilderness Rescue", "value": "mountain"},
                {"label": "Disaster or Individual Tracking", "value": "disaster"}
            ],
            value="reset",
            labelStyle={"display": "block"}
        )
    ]),

    html.Hr(),

    # ========================================================
    # INTERACTIVE DATA TABLE
    # ========================================================
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict("records"),

        # Make it client-friendly
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],

        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left", "padding": "6px"},
        style_header={"fontWeight": "bold"}
    ),

    html.Br(),
    html.Hr(),

    # Chart + Map side by side
    html.Div(
        style={"display": "flex", "gap": "20px"},
        children=[
            html.Div(id="graph-id", style={"flex": "1"}),
            html.Div(id="map-id", style={"flex": "1"})
        ]
    )
])


# ============================================================
# CALLBACK 1: FILTER DATA TABLE
# ============================================================

@app.callback(
    Output("datatable-id", "data"),
    Input("filter-type", "value")
)
def update_dashboard(filter_type):
    """
    This function runs whenever the user changes the rescue filter.
    It sends the correct MongoDB query and updates the data table.
    """

    # Breed lists from dashboard specifications
    water_breeds = [
        "Labrador Retriever Mix",
        "Chesapeake Bay Retriever",
        "Newfoundland"
    ]

    mountain_breeds = [
        "German Shepherd",
        "Alaskan Malamute",
        "Old English Sheepdog",
        "Siberian Husky",
        "Rottweiler"
    ]

    disaster_breeds = [
        "Doberman Pinscher",
        "German Shepherd",
        "Golden Retriever",
        "Bloodhound",
        "Rottweiler"
    ]

    query = {}

    if filter_type == "water":
        query = {
            "breed": {"$in": water_breeds},
            "sex_upon_outcome": {"$in": ["Intact Female", "Intact Male"]},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }

    elif filter_type == "mountain":
        query = {
            "breed": {"$in": mountain_breeds},
            "sex_upon_outcome": {"$in": ["Intact Female", "Intact Male"]},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }

    elif filter_type == "disaster":
        query = {
            "breed": {"$in": disaster_breeds},
            "sex_upon_outcome": {"$in": ["Intact Female", "Intact Male"]},
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }

    records = db.read(query, {"_id": 0})
    dff = pd.DataFrame.from_records(records)

    if dff is None or dff.empty:
        return []

    return dff.to_dict("records")


# ============================================================
# CALLBACK 2: PIE CHART (Breed Distribution)
# ============================================================

@app.callback(
    Output("graph-id", "children"),
    Input("datatable-id", "derived_virtual_data")
)
def update_graph(viewData):

    if viewData is None or len(viewData) == 0:
        return html.Div("No data available.")

    dff = pd.DataFrame(viewData)

    if "breed" not in dff.columns:
        return html.Div("Breed column not found.")

    breed_counts = dff["breed"].value_counts().reset_index()
    breed_counts.columns = ["breed", "count"]

    fig = px.pie(
        breed_counts,
        names="breed",
        values="count",
        title="Breed Distribution of Filtered Results"
    )

    return dcc.Graph(figure=fig)


# ============================================================
# CALLBACK 3: GEOLOCATION MAP
# ============================================================

@app.callback(
    Output("map-id", "children"),
    [Input("datatable-id", "derived_virtual_data"),
     Input("datatable-id", "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):

    if viewData is None or len(viewData) == 0:
        return html.Div("No data available for map.")

    dff = pd.DataFrame(viewData)

    if selected_rows is None or len(selected_rows) == 0:
        row = 0
    else:
        row = selected_rows[0]

    lat_col = "location_lat"
    lon_col = "location_long"

    if lat_col not in dff.columns or lon_col not in dff.columns:
        return html.Div("Latitude/Longitude columns not found.")

    lat = dff.loc[row, lat_col]
    lon = dff.loc[row, lon_col]

    breed = dff.loc[row, "breed"] if "breed" in dff.columns else "Unknown"
    name = dff.loc[row, "name"] if "name" in dff.columns else "Unknown"

    return dl.Map(
        style={"width": "100%", "height": "500px"},
        center=[30.75, -97.48],  # Austin, TX
        zoom=10,
        children=[
            dl.TileLayer(),
            dl.Marker(
                position=[lat, lon],
                children=[
                    dl.Tooltip(str(breed)),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(str(name))
                    ])
                ]
            )
        ]
    )


# ============================================================
# RUN SERVER
# ============================================================
print("Data loaded:", len(df))
print("Columns:", df.columns)
app.run_server(mode="inline", debug=False)

Data loaded: 10002
Columns: Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Feb/2026 21:26:01] "GET /_alive_d6a42a47-3ea4-417e-a99d-2985928d6e83 HTTP/1.1" 200 -


In [3]:
print("Total records:", len(db.read({}, {"_id": 0})))

print("Water filter:", len(db.read({
    "breed": {"$in": ["Labrador Retriever Mix"]},
    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
}, {"_id": 0})))

Total records: 10002
Water filter: 249


In [4]:
# ---- PROOF THAT FILTERS WORK (for README screenshots) ----
# Total records
print("Total records:", len(db.read({}, {"_id": 0})))

# Water Rescue (26 to 156 weeks = ~6 months to 3 years)
water_query = {
    "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
    "sex_upon_outcome": {"$in": ["Intact Female", "Intact Male"]},
    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
}
print("Water Rescue records:", len(db.read(water_query, {"_id": 0})))

# Mountain/Wilderness Rescue
mountain_query = {
    "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
    "sex_upon_outcome": {"$in": ["Intact Female", "Intact Male"]},
    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
}
print("Mountain/Wilderness records:", len(db.read(mountain_query, {"_id": 0})))

# Disaster/Tracking
disaster_query = {
    "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
    "sex_upon_outcome": {"$in": ["Intact Female", "Intact Male"]},
    "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
}
print("Disaster/Tracking records:", len(db.read(disaster_query, {"_id": 0})))

# Reset proof (same as total)
print("Reset records:", len(db.read({}, {"_id": 0})))

Total records: 10002
Water Rescue records: 35
Mountain/Wilderness records: 5
Disaster/Tracking records: 6
Reset records: 10002
